In [2]:
import smtplib
import json
from tabulate import tabulate
from email.mime.text import MIMEText
import yfinance as yf
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

In [3]:
oversold = 25
period = 2
timeframe = "1d"

In [4]:
'''
The backtesting results were obtained using TradingView for the following parameters:
oversold <= 25
time period: 01-01-2015 to 30-05-2024
Only the best performing stocks were selected!
'''

backtesting_result = pd.read_csv("./ifr2_best_stocks_backtesting.csv")
backtesting_result

,Ticker,Total Trades,Profitable %,Gain Factor,Max Drawdown,Mean Transaction,Total Candles
0,CPLE6,237,71.31,2.521,38.86,1.09,5
1,TAEE11,220,76.35,2.576,14.87,0.82,5
2,VIVT3,248,77.42,2.769,13.94,0.99,4
3,ELET3,238,64.71,1.165,40.14,0.42,5
4,SLCE3,220,72.73,1.701,30.12,0.91,5
...,...,...,...,...,...,...,...
67,HYPE3,224,69.64,1.466,28.37,0.48,5
68,ALPA3,227,70.48,1.311,51.98,0.76,4
69,SBSP3,227,69.60,1.244,47.11,0.41,5
70,PETR3,218,68.35,1.356,66.05,0.82,5


In [5]:
# Define a custom scaling function
def custom_scale(series, scale_factor=10):
    min_val = series.min()
    max_val = series.max()
    return ((series - min_val) / (max_val - min_val) * scale_factor) + 1

# Apply the custom scaling function
backtesting_result['Total Candles Scaled'] = custom_scale(backtesting_result['Total Candles'])
backtesting_result["Greatness Factor"] = backtesting_result["Gain Factor"] * backtesting_result["Profitable %"]
backtesting_result["Greatness Factor Scaled"] = custom_scale(backtesting_result["Greatness Factor"], 50)
backtesting_result.sort_values(by="Greatness Factor", inplace=True)
backtesting_result.reset_index(inplace=True)

In [6]:
# Create 3D scatter plot
fig = px.scatter_3d(backtesting_result, x='Profitable %', y='Gain Factor', z='Total Trades', color="Max Drawdown", size="Total Candles Scaled", text="Ticker", size_max=30)

# Show plot
fig.show()

In [7]:
today = datetime.today()
i_date = (today - timedelta(days=30)).strftime('%Y-%m-%d')
f_date = today.strftime('%Y-%m-%d')
selected_stocks = [ticker + ".SA" for ticker in backtesting_result["Ticker"]]

# tickers = [ 
#     "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "MGLU3.SA", "ABEV3.SA", "BBAS3.SA", "B3SA3.SA",
#     "JBSS3.SA", "USIM5.SA", "WEGE3.SA", "SANB11.SA", "SUZB3.SA", "BRKM5.SA", "ELET3.SA",
#     "GGBR4.SA", "CEAB3.SA", "CSNA3.SA", "ITSA4.SA", "RDOR3.SA", "BEEF3.SA", "LREN3.SA",
#     "MULT3.SA", "HAPV3.SA", "EQTL3.SA", "TAEE11.SA", "EGIE3.SA", "KLBN11.SA", "TIMS3.SA", "BRFS3.SA",
#     "RAIL3.SA", "NTCO3.SA", "IGTI11.SA", "MRFG3.SA", "UGPA3.SA", "LIGT3.SA", "ARZZ3.SA", "CCRO3.SA",
#     "FLRY3.SA", "LWSA3.SA", "CYRE3.SA", "SOJA3.SA", "ENEV3.SA", "CPFE3.SA", "ALPA4.SA", "AESB3.SA",
#     "VIVA3.SA", "TOTS3.SA", "RADL3.SA", "RENT3.SA", "AALR3.SA", "IRBR3.SA", "GOLL4.SA",
#     "EZTC3.SA", "COGN3.SA", "POSI3.SA", "QUAL3.SA", "TEND3.SA", "JHSF3.SA",
#     "TUPY3.SA", "SAPR11.SA", "CMIG4.SA", "SIMH3.SA", "BRIT3.SA", "PETZ3.SA",
#     "BMOB3.SA", "BPAN4.SA", "ENJU3.SA", "MOVI3.SA", "TGMA3.SA", "BBSE3.SA", "MRVE3.SA",
#     "EMBR3.SA", "CSAN3.SA", "CIEL3.SA",
#     "NEOE3.SA", "ARZZ3.SA", "CVCB3.SA", "SBSP3.SA", "POMO4.SA", "PRIO3.SA",
#     "PSSA3.SA", "ECOR3.SA", "LOGG3.SA", "CXSE3.SA", "YDUQ3.SA", "PCAR3.SA", "AMER3.SA", 
#     "HYPE3.SA", "GOAU4.SA", "ENGI11.SA", "CRFB3.SA", "BRAP4.SA", "BPAC11.SA", "AZUL4.SA"
# ]

# Download data using yfinance
data = yf.download(selected_stocks, start=i_date, end=f_date, interval=timeframe)

[*********************100%%**********************]  72 of 72 completed


In [8]:
close = data["Adj Close"].iloc[-1]
var = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2] * 100

# Wilder method for RSI
# This calculation was made to match TradingView's RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
    avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

rsi = calculate_rsi(data['Close'], period).iloc[-1]
# rsi = vbt.RSI.run(data["Adj Close"], window=2).rsi.iloc[-1].loc[(2)]

df = pd.DataFrame({
    'Close': close,
    'Var': var,
    'RSI': rsi,
}).reset_index(drop=False)

df["Ticker"] = df["Ticker"].str.replace(".SA", "") # Remove ".SA" suffix

In [9]:
# Filtering based on oversold condition
oversold_df = df[df["RSI"] <= oversold]
# Merging to get the previous calculated columns
oversold_df = oversold_df.merge(backtesting_result, how="left", on="Ticker") 
oversold_df.drop(["Total Candles Scaled", "Greatness Factor Scaled"], axis="columns", inplace=True)
oversold_df.sort_values(by="Greatness Factor", ascending=False, inplace=True)
oversold_df.reset_index(inplace=True, drop=True)

In [10]:
# Create the scatter plot
fig = px.scatter(oversold_df, x='Greatness Factor', y='RSI', text='Ticker', color="Var", size_max=30)

# Update layout to improve appearance
fig.update_traces(textposition='top center')
fig.update_layout(title='RSI of Different Tickers',
                  yaxis= dict(title='RSI', range=[min(oversold_df['RSI']) - 5, max(oversold_df['RSI']) + 10], zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey'),
                  xaxis=dict(title='Greatness Factor', range=[min(oversold_df["Greatness Factor"] - 40), max(oversold_df['Greatness Factor']) + 40], zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey'),
                  height=600)

# Show plot
fig.show()

In [11]:
oversold_df

,Ticker,Close,Var,RSI,index,Total Trades,Profitable %,Gain Factor,Max Drawdown,Mean Transaction,Total Candles,Greatness Factor
0,REDE3,6.820000,-1.015961,18.837715,49,166,77.11,6.000,22.34,2.20,4,462.66000
1,ENGI4,7.790000,-1.517066,10.056356,42,223,79.37,4.667,15.07,2.54,4,370.41979
2,TRPL3,33.799999,-2.028988,4.560324,52,193,83.94,3.439,18.58,1.69,4,288.66966
3,VIVT3,43.299999,-3.045232,0.925546,2,248,77.42,2.769,13.94,0.99,4,214.37598
4,PTNT4,6.970000,-0.428574,12.446942,48,219,78.08,2.548,42.63,1.87,4,198.94784
5,TAEE11,34.279999,-0.810193,4.548767,1,220,76.35,2.576,14.87,0.82,5,196.67760
6,CPLE6,9.080000,-1.625131,2.052891,0,237,71.31,2.521,38.86,1.09,5,179.77251
7,ENGI3,14.020000,-0.142447,20.291964,41,194,73.23,2.430,24.43,1.66,4,177.94890
8,MNPR3,10.700000,-5.141843,0.167534,37,239,76.99,2.209,47.55,4.38,4,170.07091
9,UCAS3,2.270000,-2.991450,9.856334,50,242,73.97,1.964,30.92,1.52,5,145.27708


In [15]:
"asd@gmail,basdbas@asd".split(',')

['asd@gmail', 'basdbas@asd']

In [17]:
ascii_table = tabulate(oversold_df[["Ticker", "Var", "RSI", "Greatness Factor"]], headers='keys', tablefmt='markdown')

subject = "IFR2 Screener - Daily Collection"
body = ascii_table

with open("./.secrets/credentials.json", "r") as file:
   credentials = json.load(file)

sender = credentials["sender"]
recipients = credentials["recipients"]
password = credentials["password"]

def send_email(subject, body, sender, recipients, password):
   msg = MIMEText(body)
   msg['Subject'] = subject
   msg['From'] = sender
   msg['To'] = recipients
   with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
      smtp_server.login(sender, password)
      smtp_server.sendmail(sender, recipients.split(','), msg.as_string())
   print("Message sent!")

send_email(subject, body, sender, recipients, password)

Message sent!
